In [ ]:
from __future__ import annotations

import pandas as pd
from carps.analysis.gather_data import normalize_logs, rename_legacy

original_optimizers = {
    "blackbox": ["RandomSearch", "SMAC3-BlackBoxFacade", "Nevergrad-CMA-ES"],
    "multi-objective": ["RandomSearch", "Optuna-MO-TPE", "Nevergrad-DE"],
    "multi-fidelity": ["SMAC3-Hyperband", "SMAC3-MultiFidelityFacade", "DEHB"],
    "multi-fidelity-objective": ["RandomSearch", "SMAC3-MOMF-GP", "Nevergrad-DE"],
}

original_optimizers_only = True
origins = {
    "full": "full",
    "dev": "discrepancy",
    "test": "discrepancy",
}

df = pd.read_parquet("../rundata.parquet")
print(df["scenario"].unique())
print(df.groupby("scenario").apply(lambda gdf: gdf["optimizer_id"].unique()))
if original_optimizers_only:
    D = []
    for scenario, gdf in df.groupby("scenario"):
        gdf = gdf[gdf["optimizer_id"].isin(original_optimizers[scenario])]
        D.append(gdf)
    df = pd.concat(D).reset_index(drop=True)
df = rename_legacy(df)
df = normalize_logs(df)
df["origin"] = df["set"].apply(lambda x: origins[x])

In [ ]:
from autorank._util import get_sorted_rank_groups
from carps.analysis.plot_ranking import calc_critical_difference

rank_results = {}
for (scenario, set_id), gdf in df.groupby(by=["task_type", "set"]):
    print(scenario, set_id)
    print(gdf["optimizer_id"].unique())
    perf_col: str = "trial_value__cost_inc_norm"
    identifier = f"{scenario}_{set_id}"
    result = calc_critical_difference(gdf, identifier=identifier, figsize=(8, 3), perf_col=perf_col)
    sorted_ranks, names, groups = get_sorted_rank_groups(result, reverse=False)
    avg_performance = gdf.groupby("optimizer_id")[perf_col].mean().sort_values()
    rank_results[(scenario, set_id)] = (result, avg_performance)

In [ ]:
key_order = "meanrank"
from rich import inspect

# R = {k: v["mearn_rank"] for k, v in rank_results.items()}
R = []
for i, (k, v) in enumerate(rank_results.items()):
    rank_result, avg_performance = v
    # inspect(rank_result)
    perf = pd.DataFrame([rank_result.rankdf["meanrank"], avg_performance]).T
    perf = perf.rename(columns={"trial_value__cost_inc_norm": "trial_value__cost_inc_norm_avg"})
    perf["scenario"] = k[0]
    perf["set_id"] = k[1]
    perf["is_not_significant"] = rank_result.pvalue >= rank_result.alpha
    perf.index.name = "optimizer_id"
    perf = perf.reset_index()
    # perf.index = [i]
    # del perf["trial_value__cost_inc_norm_avg"]
    perf["order"] = perf[key_order].rank(method="max", numeric_only=True).astype(int)
    # d = perf.melt(id_vars=["scenario", "set_id"], var_name="optimizer_id", value_name="meanrank")
    # d["order"] = d.rank(method="max", numeric_only=True).astype(int)
    # R.append(d)
    R.append(perf)
    # break

df_rank = pd.concat(R).reset_index(drop=True)
df_rank

In [ ]:
import numpy as np
import pandas as pd

fn_template = "ranks_per_set_{scenario}.csv"

decimal_places = 2

final_str = r"""
\begin{{table}}[h]
    \caption{{{caption}}}
    \label{{{label}}}
    \centering
    %\resizebox{{0.4\textwidth}}{{!}}{{
    {table_string}
    %}}
\end{{table}}
"""

float_format = lambda x: ("{:0." + str(decimal_places) + "f}").format(x) if not np.isnan(x) else "-"


for scenario, gdf in df_rank.groupby("scenario"):
    fn = fn_template.format(scenario=scenario)
    sorter = gdf[gdf["set_id"]=="dev"].sort_values(key_order)["optimizer_id"].to_list()

    R = gdf.pivot_table(index="set_id", columns="optimizer_id", values="order").map(int)
    origs = original_optimizers[scenario]
    origs.sort(key=lambda x: sorter.index(x))
    cols = origs + [c for c in R.columns if c not in original_optimizers[scenario]]
    R = R[cols]


    MR = gdf.pivot_table(index="set_id", columns="optimizer_id", values="meanrank").map(lambda x: f"{x:.2f}" if not isinstance(x, str) else x)
    MR = MR[cols]
    for i, ((_idx, row), (_idx2, row2)) in enumerate(zip(MR.iterrows(), R.iterrows(), strict=False)):
        for j in range(len(row)):
            row.iloc[j] = row.iloc[j] + f" ({int(row2.iloc[j])})"

    def get_significance_str(dataframe) -> str:
        if dataframe["is_not_significant"].all():
            return "no"
        return "yes"

    MR["significant"] = gdf.groupby("set_id").apply(get_significance_str)

    table_str = MR.to_latex(float_format=float_format, na_rep="-").strip()
    caption = f"Mean Ranking for Scenario {scenario}"
    if gdf["is_not_significant"].all():
        caption += " (non-significant)"
    label = f"tab:ranking_validation_{scenario}"
    table_str = table_str.replace("_", r"\_")
    table_str = final_str.format(table_string=table_str, label=label, caption=caption)

    with open(fn + ".tex", "w") as file:
        file.write(table_str)
    print(table_str)
